In [ ]:
#Download and Extract the Dataset:
!wget https://ai2-s2-mslr.s3.us-west-2.amazonaws.com/mslr_data.tar.gz
!tar -xvf mslr_data.tar.gz

#Delete the Cochrane dataset and any other unwanted files:
!rm -r mslr_data/cochrane/
!rm mslr_data.tar.gz*

#Move the ms2 directory up one level and remove the parent mslr_data directory:
!mv mslr_data/ms2 ./
!rm -r mslr_data/
!rm -r sample_data/

_______

In [1]:
# sure the GPUs are working 
import tensorflow as tf

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")
#import 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration, AutoConfig
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

import evaluate
from pprint import pprint  # Makes longer output readable without horizontal scrolling

import csv


# Load the data- test inputs
df = pd.read_csv('ms2/test-inputs.csv')

# Group by ReviewID and concatenate all Abstracts
df = df.groupby('ReviewID').agg({
    'Abstract': ' '.join
}).reset_index()

df.head()



TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.0


,ReviewID,Abstract
0,9712593,A r and omized controlled prospect i ve clinic...
1,10103327,OBJECTIVES To measure the proportion of childr...
2,10567628,Patients with chronic obstructive pulmonary di...
3,10732935,It is often necessary to adjust a patient ’s s...
4,10796140,Previous studies have found that there is a co...


In [ ]:
# # Replace 'your_review_id' with the actual ReviewID you're looking for
# your_review_id = 9712593

# # Select the row where 'ReviewID' matches 'your_review_id'
# target_text = df.loc[df['ReviewID'] == your_review_id, 'Abstract'].values[0]

# print(target_text)

In [ ]:
#tokenize

# Initialize the Pegasus tokenizer and model
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")

# Tokenize the inputs
inputs = tokenizer(df['Abstract'].tolist(), max_length=1024, 
                    truncation=True, padding="max_length", return_tensors="pt")  # Note return_tensors="pt" for PyTorch


In [ ]:
# Define the batch size
batch_size = 5

# Initialize the results list
results = []

# Generate summaries in batches of 5
for i in range(0, len(inputs['input_ids']), batch_size):
    input_ids_batch = inputs['input_ids'][i:i+batch_size]
    review_ids_batch = df['ReviewID'][i:i+batch_size].tolist()  # Get the corresponding ReviewIDs
    summary_ids = model.generate(input_ids_batch,
                                 num_beams=2,
                                 no_repeat_ngram_size=2,
                                 min_length=10,
                                 max_length=800,
                                 early_stopping=True)
    batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    # Append each summary with its ReviewID to the results list
    for review_id, summary in zip(review_ids_batch, batch_summaries):
        results.append({'ReviewID': review_id, 'Summary': summary})

# Convert the results to a DataFrame
summaries_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
output_file = '/Users/faridgholitabar/Coding/266_final_proj/test-prediction.csv'
summaries_df.to_csv(output_file, index=True)
print(f"Saved summaries to {output_file}")

_______

***Run the same code on the dev dataset***

In [ ]:
# Load the data - test inputs
df_dev = pd.read_csv('ms2/dev-inputs.csv')

# Group by ReviewID and concatenate all Abstracts
df_dev = df_dev.groupby('ReviewID').agg({
    'Abstract': ' '.join
}).reset_index()

print(df_dev.head())

# Initialize the Pegasus tokenizer and model
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")

# Tokenize the inputs
inputs = tokenizer(df_dev['Abstract'].tolist(), max_length=1024, 
                    truncation=True, padding="max_length", return_tensors="pt")  # Note return_tensors="pt" for PyTorch

# Define the batch size
batch_size = 5

# Initialize the results list
results = []

# Generate summaries in batches
for i in range(0, len(inputs['input_ids']), batch_size):
    input_ids_batch = inputs['input_ids'][i:i+batch_size]
    review_ids_batch = df_dev['ReviewID'][i:i+batch_size].tolist()  # Get the corresponding ReviewIDs
    summary_ids = model.generate(input_ids_batch,
                                 num_beams=2,
                                 no_repeat_ngram_size=2,
                                 min_length=10,
                                 max_length=800,
                                 early_stopping=True)
    batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    # Append each summary with its ReviewID to the results list
    for review_id, summary in zip(review_ids_batch, batch_summaries):
        results.append({'ReviewID': review_id, 'Summary': summary})

# Convert the results to a DataFrame
summaries_df_dev = pd.DataFrame(results)

# Save the DataFrame to a CSV file
output_file_dev = 'dev-prediction.csv'
summaries_df_dev.to_csv(output_file_dev, index=True)
print(f"Saved summaries to {output_file_dev}")


In [7]:
# Load the data - test inputs
df_dev = pd.read_csv('ms2/dev-inputs.csv')

# Group by ReviewID and concatenate all Abstracts
df_dev = df_dev.groupby('ReviewID').agg({
    'Abstract': ' '.join
}).reset_index()

df_dev.head()

,ReviewID,Abstract
0,9470890,"A clinical trial , aim ed at secondary prevent..."
1,9678145,Reports in the surgical literature are conflic...
2,9755013,"In the majority of patients with neck pain , s..."
3,9876364,The authors evaluated a six-session interactiv...
4,10716002,This paper reports the 5-year findings of a mu...


In [8]:
df_dev.shape

(2021, 2)

In [3]:
df.shape

(1667, 2)

In [6]:
df_train = pd.read_csv('ms2/train-inputs.csv')
df_train.shape

(323608, 5)